In [1]:
# Transformers installation
! pip install -U transformers datasets fsspec kaggle evaluate optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 

# Fine-tune a pretrained model

## Prepare a dataset

Go to [kaggle.com/settings](https://) and create a new token present under the API Tab. This will generate and download a kaggle.json file. Upload it below.

In [2]:
#Upload kaggle.json
from google.colab import files

uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
#Move kaggle.json to /root/.kaggle folder and provide read, write and execute permissions
!mkdir -p ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
#Download the Jigsaw Toxic Comment Classification Dataset
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

  0% 0.00/52.6M [00:00<?, ?B/s]
100% 52.6M/52.6M [00:00<00:00, 973MB/s]


In [5]:
#Unzip the dataset
!unzip -o jigsaw-toxic-comment-classification-challenge.zip -d jigsaw-toxic-comment-classification
!cd jigsaw-toxic-comment-classification
!unzip -o jigsaw-toxic-comment-classification/train.csv.zip -d jigsaw-toxic-comment-classification
!unzip -o jigsaw-toxic-comment-classification/test.csv.zip -d jigsaw-toxic-comment-classification
!unzip -o jigsaw-toxic-comment-classification/test_labels.csv.zip -d jigsaw-toxic-comment-classification

Archive:  jigsaw-toxic-comment-classification-challenge.zip
  inflating: jigsaw-toxic-comment-classification/sample_submission.csv.zip  
  inflating: jigsaw-toxic-comment-classification/test.csv.zip  
  inflating: jigsaw-toxic-comment-classification/test_labels.csv.zip  
  inflating: jigsaw-toxic-comment-classification/train.csv.zip  
Archive:  jigsaw-toxic-comment-classification/train.csv.zip
  inflating: jigsaw-toxic-comment-classification/train.csv  
Archive:  jigsaw-toxic-comment-classification/test.csv.zip
  inflating: jigsaw-toxic-comment-classification/test.csv  
Archive:  jigsaw-toxic-comment-classification/test_labels.csv.zip
  inflating: jigsaw-toxic-comment-classification/test_labels.csv  


In [6]:
#Load dataset
from datasets import load_dataset

dataset = load_dataset("google/jigsaw_toxicity_pred", data_dir="jigsaw-toxic-comment-classification", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.37k [00:00<?, ?B/s]

jigsaw_toxicity_pred.py:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/159571 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/63978 [00:00<?, ? examples/s]

In [7]:
#Create a validation dataset
from datasets import DatasetDict
train_dataset = dataset["train"]
validation_dataset = dataset["test"].select(range(2000))
final_test_dataset = dataset["test"].select(
    range(2000, len(dataset["test"]))
)

raw_datasets = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": final_test_dataset
})

In [8]:
#Preprocess dataset by adding a new label that tells if comment is toxic or not and then tokenize them.
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def convert_to_binary_classifier(comments):
  label_columns = ['toxic',
  'severe_toxic',
  'obscene',
  'threat',
  'insult',
  'identity_hate']
  is_toxic_labels = [any(comments[label][i] == 1 for label in label_columns) for i in range(len(comments['comment_text']))]
  tokenized_output = tokenizer(comments["comment_text"], padding="max_length", truncation=True)
  tokenized_output["labels"] = [1 if is_toxic else 0 for is_toxic in is_toxic_labels]
  return tokenized_output

tokenized_datasets = raw_datasets.map(convert_to_binary_classifier, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/159571 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/61978 [00:00<?, ? examples/s]

### Setup Evaluation Metrics

In [9]:
#Setup evaluation metrics
import numpy as np
import evaluate

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

In [10]:
#For this training, primary objective is to maximize F1 score.
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy_result = accuracy_metric.compute(predictions=predictions, references=labels)
    accuracy = accuracy_result.get("accuracy", 0.0)
    f1_result = f1_metric.compute(predictions=predictions, references=labels)
    f1 = f1_result.get("f1", 0.0)
    return {"f1": f1, "accuracy" : accuracy}

In [11]:
from transformers import AutoModelForSequenceClassification
def model_init(trial):
  return AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [12]:
def compute_objective(metrics):
    print(metrics)
    return metrics['eval_f1']

In [13]:
#Enable fp16 for faster training due to GPU constraints and evaluate after each epoch
from transformers import TrainingArguments

hp_search_training_args = TrainingArguments(output_dir="./results",
    eval_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
    fp16=True)

In [14]:
#Subset of the complete dataset for hyperparameter search.
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(20000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)

### Trainer

In [15]:
from transformers import Trainer
hp_search_trainer = Trainer(
    args=hp_search_training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    model_init=model_init,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Find the optimal hyperparameters such that the F1 score is maximized.

In [ ]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1,2),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "weight_decay": trial.suggest_float("weight_decay", 0.001, 0.1),
    }

best_trials = hp_search_trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=10,
    compute_objective=compute_objective,
)
best_trials

[I 2025-06-11 04:39:29,344] A new study created in memory with name: no-name-acd9c96b-d7a6-49a2-b465-840d2b9f3909
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.121200,0.150592,0.683371,0.930500
2,0.053800,0.222534,0.694878,0.931500


{'eval_loss': 0.15059153735637665, 'eval_f1': 0.683371298405467, 'eval_accuracy': 0.9305}
{'eval_loss': 0.22253426909446716, 'eval_f1': 0.6948775055679287, 'eval_accuracy': 0.9315}


[I 2025-06-11 04:48:27,265] Trial 0 finished with value: 0.6948775055679287 and parameters: {'learning_rate': 4.910748967246961e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'weight_decay': 0.025747150559257862}. Best is trial 0 with value: 0.6948775055679287.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.132400,0.187791,0.691630,0.930000
2,0.074300,0.246111,0.684211,0.928000


...

[I 2025-06-11 05:42:41,233] Trial 8 pruned. 
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.131500,0.236782,0.679487,0.925000


{'eval_loss': 0.2367822527885437, 'eval_f1': 0.6794871794871795, 'eval_accuracy': 0.925}


[I 2025-06-11 05:47:11,558] Trial 9 pruned. 


BestRun(run_id='0', objective=0.6948775055679287, hyperparameters={'learning_rate': 4.910748967246961e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'weight_decay': 0.025747150559257862}, run_summary=None)

###FineTune Model

In [17]:
from transformers import TrainingArguments, Trainer

final_training_args = TrainingArguments(
    output_dir="./final_model_toxicity_classification",
    eval_strategy="epoch",
    logging_dir="./final_model_logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
    fp16=True,
    # Apply the best hyperparameters directly from best_trials
    learning_rate=4.910748967246961e-05,
    num_train_epochs=2,
    per_device_train_batch_size=32,
    weight_decay=0.025747150559257862,
)

In [22]:
final_trainer = Trainer(
    model=AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2),
    args=final_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
final_trainer.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.095900,0.166347,0.724771,0.940000
2,0.052700,0.270186,0.677551,0.921000


TrainOutput(global_step=9974, training_loss=0.07432684660292542, metrics={'train_runtime': 3904.4364, 'train_samples_per_second': 81.738, 'train_steps_per_second': 2.555, 'total_flos': 4.227591054187315e+16, 'train_loss': 0.07432684660292542, 'epoch': 2.0})

###Save Model and Tokenizer

In [24]:
final_trainer.save_model(final_training_args.output_dir)

In [26]:
tokenizer.save_pretrained(final_training_args.output_dir)

('./final_model_toxicity_classification/tokenizer_config.json',
 './final_model_toxicity_classification/special_tokens_map.json',
 './final_model_toxicity_classification/vocab.txt',
 './final_model_toxicity_classification/added_tokens.json',
 './final_model_toxicity_classification/tokenizer.json')

###Evaluate on Test Set

In [45]:
test_results = final_trainer.evaluate(tokenized_datasets["test"])
test_results

{'eval_loss': 0.18162037432193756,
 'eval_f1': 0.7053535489064806,
 'eval_accuracy': 0.9291361450837394,
 'eval_runtime': 260.3667,
 'eval_samples_per_second': 238.041,
 'eval_steps_per_second': 29.758,
 'epoch': 2.0}

###Uplod Model to Huggingface Hub

In [27]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `token_1` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate 

In [30]:
final_trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/godsofheaven/final_model_toxicity_classification/commit/cfa20922f2ac036b2b5edd81f5cdb0cfe1c47e9c', commit_message='End of training', commit_description='', oid='cfa20922f2ac036b2b5edd81f5cdb0cfe1c47e9c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/godsofheaven/final_model_toxicity_classification', endpoint='https://huggingface.co', repo_type='model', repo_id='godsofheaven/final_model_toxicity_classification'), pr_revision=None, pr_num=None)

###Inference

In [35]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="godsofheaven/final_model_toxicity_classification")

Device set to use cuda:0


In [48]:
user_input = input("Enter your comment here: ")
prediction = pipe(user_input)
pred = prediction[0]

# Interpret the prediction
predicted_label_index = int(pred['label'].split('_')[1])
predicted_class = "UNSAFE" if predicted_label_index == 1 else "SAFE"
confidence = pred['score']

print("\n--- Pipeline Prediction for Single Input ---")
print(f"Text: '{user_input}'")
print(f"  Prediction: {predicted_class} (Confidence: {confidence:.4f})")
print("-" * 30)

Enter your comment here: You are an idiot and should be banned!

--- Pipeline Prediction for Single Input ---
Text: 'You are an idiot and should be banned!'
  Prediction: UNSAFE (Confidence: 0.9971)
------------------------------
